In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
for dirname, _, filenames in os.walk('/kaggle/output'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/enron-dataset-cleaned/emails.csv


# Step-1 : Initial Setup

## Imports

In [2]:
import os
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser

import spacy
from nltk.corpus import stopwords

# for plotting
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

## Downloads

In [3]:
# # import nltk
# # nltk.download('stopwords')
# #python -m spacy download en
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip
# os.listdir('/kaggle/working')

--2020-08-04 18:39:48--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  5.97MB/s    in 2.6s    

2020-08-04 18:39:51 (5.97 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classi

  inflating: mallet-2.0.8/class/cc/mallet/fst/FeatureTransducer$Transition.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/FeatureTransducer$TransitionIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/FeatureTransducer.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/HMM$Incrementor.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/HMM$State.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/HMM$TransitionIterator.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/HMM$WeightedIncrementor.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/HMM.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/HMMTrainerByLikelihood.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/InstanceAccuracyEvaluator.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/LabelDistributionEvaluator.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/MaxLattice.class  
  inflating: mallet-2.0.8/class/cc/mallet/fst/MaxLatticeDefault$1.class  
  inflating: mallet-2.0.8/clas

  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizable$ByGradient.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizable$ByGradientValue.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizable$ByHessian.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizable$ByValue.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizable$ByVotedPerceptron.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizable.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/OptimizableCollection$ByGradientValue.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/OptimizableCollection.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/OptimizationException.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizer$ByBatches.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/Optimizer.class  
  inflating: mallet-2.0.8/class/cc/mallet/optimize/OptimizerEvaluator$ByBatchGradient.class  
  inflatin

  inflating: mallet-2.0.8/class/cc/mallet/types/tests/TestSerializable$WriteMe.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/tests/TestSerializable.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/tests/TestSparseMatrixn.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/tests/TestSparseVector.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/tests/TestToken.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/Token.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/TokenSequence.class  
  inflating: mallet-2.0.8/class/cc/mallet/types/Vector.class  
   creating: mallet-2.0.8/class/cc/mallet/util/
  inflating: mallet-2.0.8/class/cc/mallet/util/Addable.class  
  inflating: mallet-2.0.8/class/cc/mallet/util/ArrayListUtils.class  
  inflating: mallet-2.0.8/class/cc/mallet/util/ArrayUtils.class  
  inflating: mallet-2.0.8/class/cc/mallet/util/BshInterpreter.class  
  inflating: mallet-2.0.8/class/cc/mallet/util/BulkLoader.class  
  inflating: mallet-2.0.8

  inflating: mallet-2.0.8/src/cc/mallet/classify/MaxEntPRTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/MaxEntTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/MCMaxEnt.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/MCMaxEntTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/MostFrequentClassAssignmentTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/MostFrequentClassifier.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/NaiveBayes.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/NaiveBayesEMTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/NaiveBayesTrainer.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/package.html  
  inflating: mallet-2.0.8/src/cc/mallet/classify/PRAuxClassifier.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/PRAuxClassifierOptimizable.java  
  inflating: mallet-2.0.8/src/cc/mallet/classify/RandomAssignmentTrainer.java  
  inflating: mallet-2.0.8/src/cc/ma

  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestInference.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestListVarSet.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestLogTableFactor.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestMIntInt2ObjectMap.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestNormalFactor.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestPottsFactor.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestRandomGraphs.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestTableFactor.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestTRP.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestUndirectedModel.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestUniformFactor.java  
  inflating: mallet-2.0.8/src/cc/mallet/grmm/test/TestUniNormalFactor.java  
   creating: mallet-2.0.8/src/cc/mallet/grmm/types/
  inflating: mallet-2.0.8/src/cc/mallet/grmm/types/AbstractAss

  inflating: mallet-2.0.8/src/cc/mallet/types/InfoGain.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/Instance.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/InstanceList.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/InstanceListTUI.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/InvertedIndex.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/KLGain.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/Label.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/LabelAlphabet.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/Labeler.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/Labeling.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/Labelings.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/Labels.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/LabelSequence.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/LabelsSequence.java  
  inflating: mallet-2.0.8/src/cc/mallet/types/LabelVector.java  
  inflating: mallet-2.0.8/src/cc/mallet/ty

## Config

In [4]:
IN_PATH = '/kaggle/input/enron-dataset-cleaned'
FILE_NAME = 'emails.csv'
OUT_PATH = '/kaggle/working'

stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

mallet_path = os.path.join(OUT_PATH, './mallet-2.0.8/bin/mallet')


emails = pd.read_csv(os.path.join(IN_PATH, FILE_NAME))
# emails_subset = emails[:10000]
emails_subset = emails.sample(frac=0.02, random_state=1)

## Utils

In [5]:
def parse_raw_message(raw_message):
    '''
        Funtion for cleanning each email..
    '''
    lines = raw_message.split('\n')
    email = {}
    message = ''
    keys_to_extract = ['from', 'to']
    for line in lines:
        if ':' not in line:
            message += line.strip()
            email['body'] = message
        else:
            pairs = line.split(':')
            key = pairs[0].lower()
            val = pairs[1].strip()
            if key in keys_to_extract:
                email[key] = val
    return email

def map_to_list(emails, key):
    '''
        Helper Function for parse_into_emails to wrap things up!
    '''
    results = []
    for email in emails:
        if key not in email:
            results.append('')
        else:
            results.append(email[key])
    return results

def parse_into_emails(messages):
    '''
        Function for cleaning all emails and returning them as a dictionary
    '''
    emails = [parse_raw_message(message) for message in messages]
    return {
        'body': map_to_list(emails, 'body'),
        'to': map_to_list(emails, 'to'),
        'from_': map_to_list(emails, 'from')
    }


def sent_to_words(sentences):
    '''
        # tokenize - break down each sentence into a list of words
    '''
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

# remove stop_words, make bigrams and lemmatize
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Analysing Data

In [6]:
## Before preprocessing
print(emails_subset.shape)
emails_subset.head()

(10348, 2)


,file,message
186822,jones-t/all_documents/634.,Message-ID: <17820178.1075846925335.JavaMail.e...
308790,mann-k/all_documents/5690.,Message-ID: <29110382.1075845717882.JavaMail.e...
82383,dasovich-j/sent/423.,Message-ID: <6812040.1075843194135.JavaMail.ev...
227299,kaminski-v/var/63.,Message-ID: <21547648.1075856642126.JavaMail.e...
301824,mann-k/_sent_mail/3208.,Message-ID: <12684200.1075846107179.JavaMail.e...


In [7]:
## After preprocessing
email_df = pd.DataFrame(parse_into_emails(emails_subset.message))
print(email_df.shape)
email_df.head()

(10348, 3)


,body,to,from_
0,"It would be nice if you could be at my dinner,...",alicia.goodrow@enron.com,tana.jones@enron.com
1,Absolutely.Good point! Can Peter start to dra...,Kay Mann/Corp/Enron@ENRON,Sheila Tweed@ECT on 05/15/2001 06
2,My apologies. My schedule melted down after w...,christine.piesco@oracle.com,jeff.dasovich@enron.com
3,"Vince,UK VAR breached the limit last week.UK t...","Richard Lewis/LON/ECT@ECT, James New/LON/ECT@E...",tanya.tamarchenko@enron.com
4,Any problems/comments?AM ---------------------...,"Don Hammond/PDX/ECT@ECT, Jody Blackburn/PDX/EC...",kay.mann@enron.com


In [8]:
print(email_df.iloc[3]['body'])

Vince,UK VAR breached the limit last week.UK traders asked us to review the correlations across UK gas and power aswell as the correlations across EFA slots.We did part of the work last week.Now we'll update the correlations based on historical prices.Tanya.Richard LewisLeppard/LON/ECT@ECT, Rudy Dautel/HOU/ECT@ECT, Kirstee Hewitt/LON/ECT@ECT,Naveen Andrews/Corp/Enron@ENRON, David Port/Market Risk/Corp/Enron@ENRON, TedMurphy/HOU/ECT@ECT, Simon Hastings/LON/ECT@ECT, Paul D'Arcy/LON/ECT@ECT, AmirGhodsian/LON/ECT@ECTThanks Tanya, these are interesting results.  I am on vacation next week, sohere are my current thoughts.  I am contactable on my mobile if necessary.Gas to power correlationsI see your point about gas to power correlation only affecting VAR for theconservative 30% long term correlation, combined VAR is o1mm less thanpreviously expected - so how does this affect the limit breach?  Strictlyspeaking, we are still over our UK power limit, but the limit was set when wewere assuming

# Step-2 : LDA && Topic Modeling

## Preprocessing for LDA

In [9]:
# Convert the body of the emails to a list
data = email_df.body.values.tolist()

# Convert the list of sentence into list of words <> Tokenizing
data_words = list(sent_to_words(data))

print(data_words[3])

['vince', 'uk', 'var', 'breached', 'the', 'limit', 'last', 'week', 'uk', 'traders', 'asked', 'us', 'to', 'review', 'the', 'correlations', 'across', 'uk', 'gas', 'and', 'power', 'aswell', 'as', 'the', 'correlations', 'across', 'efa', 'slots', 'we', 'did', 'part', 'of', 'the', 'work', 'last', 'week', 'now', 'we', 'll', 'update', 'the', 'correlations', 'based', 'on', 'historical', 'prices', 'tanya', 'richard', 'lewisleppard', 'lon', 'ect', 'ect', 'rudy', 'dautel', 'hou', 'ect', 'ect', 'kirstee', 'hewitt', 'lon', 'ect', 'ect', 'naveen', 'andrews', 'corp', 'enron', 'enron', 'david', 'port', 'market', 'risk', 'corp', 'enron', 'enron', 'tedmurphy', 'hou', 'ect', 'ect', 'simon', 'hastings', 'lon', 'ect', 'ect', 'paul', 'arcy', 'lon', 'ect', 'ect', 'amirghodsian', 'lon', 'ect', 'ectthanks', 'tanya', 'these', 'are', 'interesting', 'results', 'am', 'on', 'vacation', 'next', 'week', 'sohere', 'are', 'my', 'current', 'thoughts', 'am', 'contactable', 'on', 'my', 'mobile', 'if', 'necessary', 'gas', '

In [10]:
## Creating models for <> bigram and trigram 
bigram = Phrases(data_words, min_count=5, threshold=100)
trigram = Phrases(bigram[data_words], threshold=100)

## Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

## Trigram example
print(trigram_mod[bigram_mod[data_words[3]]])

['vince', 'uk', 'var', 'breached', 'the', 'limit', 'last_week', 'uk', 'traders', 'asked', 'us', 'to', 'review', 'the', 'correlations', 'across', 'uk', 'gas', 'and', 'power', 'aswell', 'as', 'the', 'correlations', 'across', 'efa', 'slots', 'we', 'did', 'part', 'of', 'the', 'work', 'last_week', 'now', 'we', 'll', 'update', 'the', 'correlations', 'based', 'on', 'historical', 'prices', 'tanya', 'richard', 'lewisleppard', 'lon', 'ect', 'ect', 'rudy_dautel', 'hou', 'ect', 'ect', 'kirstee_hewitt', 'lon', 'ect', 'ect', 'naveen_andrews', 'corp', 'enron', 'enron', 'david_port', 'market', 'risk', 'corp', 'enron', 'enron', 'tedmurphy', 'hou', 'ect', 'ect', 'simon', 'hastings', 'lon', 'ect', 'ect', 'paul', 'arcy', 'lon', 'ect', 'ect', 'amirghodsian', 'lon', 'ect', 'ectthanks', 'tanya', 'these', 'are', 'interesting', 'results', 'am', 'on', 'vacation', 'next_week', 'sohere', 'are', 'my', 'current', 'thoughts', 'am', 'contactable', 'on', 'my', 'mobile', 'if', 'necessary', 'gas', 'to', 'power', 'correl

In [11]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[3])

['breach', 'limit', 'last', 'week', 'trader', 'ask', 'review', 'correlation', 'power', 'correlation', 'efa', 'slot', 'part', 'work', 'last', 'week', 'update', 'correlation', 'base', 'historical', 'price', 'lon', 'market', 'ect', 'ect', 'ect', 'lon', 'ectthank', 'tanya', 'interesting', 'result', 'vacation', 'sohere', 'current', 'thought', 'contactable', 'mobile', 'necessary', 'gas', 'power', 'correlationsi', 'see', 'point', 'gas', 'power', 'correlation', 'affect', 'var', 'correlation', 'combine', 'var', 'mm', 'less', 'thanpreviously', 'expect', 'affect', 'limit', 'breach', 'still', 'power', 'limit', 'limit', 'set', 'wewere', 'assume', 'gas', 'power', 'correlation', 'therefore', 'high', 'portfolio', 'var', 'suggest', 'way', 'forward', 'give', 'importance', 'spread', 'option', 'power', 'book', 'allocate', 'gas', 'power', 'book', 'share', 'var', 'also', 'understand', 'mail', 'correctly', 'imply', 'gas', 'consistent', 'correlation', 'curve', 'reduce', 'efa', 'slot', 'issue', 'exist', 'efa',

In [12]:
'''
Each time we now see a token in a text, information on its frequency is paired with it.
A word/token like contract could then be represented as (6, 3) — >(token_id, token_count).\
'''

## Creating Dictionary
id2word = corpora.Dictionary(data_lemmatized)

## Create Corpus
texts = data_lemmatized

## Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]


## Building LDA Model

In [32]:
lda_model = gensim.models.ldamodel.LdaModel(
    corpus=corpus,    # Stream of document vectors or sparse matrix of shape (num_terms, num_documents)
    id2word=id2word,  # It is used to determine the vocabulary size, as well as for debugging and topic printing.
    num_topics=6,    # The number of requested latent topics to be extracted from the training corpus.
    random_state=100, # Useful for reproducibility.
    update_every=1,   # Set to 0 for batch learning, > 1 for online iterative learning.
    chunksize=100,    # Number of documents to be used in each training chunk.
    passes=10,        # Number of passes through the corpus during training.
    alpha='auto',     # auto: Learns an asymmetric prior from the corpus
    per_word_topics=True 
    # If True, the model also computes a list of topics, sorted in descending order of most likely topics for each word,
    # along with their phi values multiplied by the feature-length (i.e. word count)
)

In [33]:
print(lda_model.print_topics())# The weights reflect how important a keyword is to that topic.


[(0, '0.018*"click" + 0.016*"image" + 0.012*"free" + 0.011*"program" + 0.011*"online" + 0.010*"internet" + 0.009*"email" + 0.009*"service" + 0.009*"offer" + 0.009*"receive"'), (1, '0.012*"thank" + 0.012*"would" + 0.010*"need" + 0.008*"deal" + 0.008*"may" + 0.008*"call" + 0.007*"change" + 0.007*"question" + 0.007*"information" + 0.007*"issue"'), (2, '0.021*"say" + 0.016*"market" + 0.014*"company" + 0.012*"power" + 0.011*"energy" + 0.010*"year" + 0.010*"state" + 0.009*"price" + 0.007*"business" + 0.005*"new"'), (3, '0.018*"get" + 0.017*"go" + 0.012*"good" + 0.011*"think" + 0.011*"time" + 0.010*"know" + 0.009*"week" + 0.008*"see" + 0.008*"want" + 0.007*"would"'), (4, '0.019*"staff" + 0.018*"invite" + 0.017*"refer" + 0.017*"transfer" + 0.016*"signature" + 0.013*"coordinate" + 0.012*"motion" + 0.011*"tonight" + 0.010*"shipper" + 0.008*"preliminary"'), (5, '0.725*"com" + 0.019*"mail" + 0.005*"txt" + 0.004*"enronxgate" + 0.003*"ca" + 0.003*"comattache" + 0.003*"find" + 0.002*"net" + 0.002*"en

In [34]:
doc_lda = lda_model[corpus]

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.144074276927963

Coherence Score:  0.4150310717984584


In [16]:
# Visualize the topics
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

pyLDAvis.display(vis)

## Mallet for Topic Modeling

In [17]:
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=6, id2word=id2word)

In [18]:
# Show Topics
print(ldamallet.show_topics(formatted=False))

[(0, [('send', 0.01743252787706269), ('attach', 0.017079382745114825), ('agreement', 0.016394495216488666), ('message', 0.014575262718575434), ('question', 0.01355863279327098), ('regard', 0.01292725210281874), ('forward', 0.012253065941827365), ('receive', 0.011525372942662072), ('copy', 0.011097318237270724), ('contact', 0.011065214134366372)]), (1, [('power', 0.023626956372183684), ('energy', 0.017227541133979247), ('state', 0.015622312675571863), ('price', 0.015464656309121137), ('market', 0.011430086567677578), ('cost', 0.010433984979647997), ('gas', 0.009344722811442985), ('utility', 0.00930889181906782), ('rate', 0.007782491543885799), ('plant', 0.007717995757610503)]), (2, [('ect', 0.023007231144370636), ('offer', 0.009995375430925754), ('email', 0.008082485495669721), ('free', 0.00787227781047675), ('day', 0.007861767426217102), ('time', 0.0072101236021188935), ('make', 0.007178592449339948), ('great', 0.0069368536113680315), ('click', 0.006810729000252249), ('receive', 0.0067

In [19]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.47751024413433196


Since Coherence Score of Mattel is good we go with Mattel.

In [20]:
##  Converting lda mallet to lda model for visualizing with pyLDAvis
model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)

# Visualize the topics with mallet model
pyLDAvis.enable_notebook(sort=True)
vis = pyLDAvis.gensim.prepare(model, corpus, id2word)
pyLDAvis.display(vis)

/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/opt/conda/lib/python3.7/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)


checking for which number of topics best suits?

In [21]:
# def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
#     """
#     Compute c_v coherence for various number of topics

#     Parameters:
#     ----------
#     dictionary : Gensim dictionary
#     corpus : Gensim corpus
#     texts : List of input texts
#     limit : Max num of topics

#     Returns:
#     -------
#     model_list : List of LDA topic models
#     coherence_values : Coherence values corresponding to the LDA model with respective number of topics
#     """
#     coherence_values = []
#     model_list = []
#     for num_topics in tqdm(range(start, limit, step)):
#         model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
#         model_list.append(model)
#         coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
#         coherence_values.append(coherencemodel.get_coherence())

#     return model_list, coherence_values

In [22]:
# # run
# model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=40, step=6)

In [23]:
# # Show graph
# limit=40; start=2; step=6;
# x = range(start, limit, step)
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [24]:
# # Print the coherence scores
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [25]:
# Select the model and print the topics
# optimal_model = model_list[4]
optimal_model = ldamallet
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=8))

[(0, '0.017*"send" + 0.017*"attach" + 0.016*"agreement" + 0.015*"message" + 0.014*"question" + 0.013*"regard" + 0.012*"forward" + 0.012*"receive"'), (1, '0.024*"power" + 0.017*"energy" + 0.016*"state" + 0.015*"price" + 0.011*"market" + 0.010*"cost" + 0.009*"gas" + 0.009*"utility"'), (2, '0.023*"ect" + 0.010*"offer" + 0.008*"email" + 0.008*"free" + 0.008*"day" + 0.007*"time" + 0.007*"make" + 0.007*"great"'), (3, '0.030*"mail" + 0.017*"good" + 0.017*"week" + 0.016*"work" + 0.016*"time" + 0.013*"call" + 0.012*"update" + 0.011*"give"'), (4, '0.021*"company" + 0.015*"business" + 0.014*"market" + 0.010*"service" + 0.009*"year" + 0.008*"trading" + 0.007*"employee" + 0.007*"report"'), (5, '0.016*"deal" + 0.015*"information" + 0.012*"change" + 0.011*"transaction" + 0.011*"contract" + 0.009*"provide" + 0.009*"process" + 0.009*"day"')]


In [26]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [36]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)
# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [ ]:
df_dominant_topic.head(10)

In [ ]:
df_dominant_topic.shape

In [ ]:
df_dominant_topic[:300].to_csv('final.csv')